## Mask R-CNN (COCO instance segmentation)

Config: **COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml**

In [1]:
# CUDA and Torch setup
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
from GPUtil import showUtilization as gpu_usage

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import common libraries
import numpy as np
import os, datetime, cv2
from matplotlib import pyplot as plt

# Import common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

# Custom Dataset
from detectron2.data.datasets import register_coco_instances

torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [ ]:
root_dir = '../../data/unzipped/facade-original-coco-segmentation/'

# Register train, valid, and test datasets
register_coco_instances("building_train_coco", {}, root_dir + "train/_annotations.coco.json", root_dir + "train")
register_coco_instances("building_valid_coco", {}, root_dir + "valid/_annotations.coco.json", root_dir + "valid")
register_coco_instances("building_test_coco", {}, root_dir + "test/_annotations.coco.json", root_dir + "test")

In [ ]:
def create_run_directory(parent_directory='./maskrcnn_coco/'):
    
    # Extract run numbers and find the highest one
    existing_dirs = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]
    run_numbers = [int(d.split('_')[1]) for d in existing_dirs if d.startswith("Run_") and d.split('_')[1].isdigit()]
    next_run_number = max(run_numbers, default=-1) + 1

    # Generate new run ID and folder
    current_datetime = datetime.datetime.now().strftime('%d_%m_%y_%H_%M')
    run_id = f"Run_{next_run_number}_{current_datetime}"
    new_run_path = os.path.join(parent_directory, run_id)
    os.makedirs(new_run_path)
    
    return new_run_path

In [ ]:
print("Initial GPU Usage")
gpu_usage()

print("GPU Usage after emptying the cache")
torch.cuda.empty_cache()
gpu_usage()

In [ ]:
# Configure Pre-trained Model
runDir = create_run_directory()

cfg = get_cfg()
cfg.MODEL.DEVICE = "cuda:3" # Set GPU
cfg.OUTPUT_DIR = runDir
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("building_train_coco",)
cfg.DATASETS.TEST = ("building_valid_coco",)
cfg.TEST.EVAL_PERIOD = 100 # do evaluation once after 100 iterations on the cfg.DATASETS.TEST (Validation set)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 1000    
cfg.SOLVER.STEPS = [] # learning rate decay
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 11  # Number of classes + background

# Custom trainer class with evaluator
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs(f"{cfg.OUTPUT_DIR}/eval/", exist_ok=True)
            output_folder = f"{cfg.OUTPUT_DIR}/eval/"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    


In [ ]:
# Train Model
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Evaluate and Use Model
cfg.MODEL.WEIGHTS = os.path.join('/home/ubuntu/code/mt-facade-seg-mask-r-cnn/maskrcnn_coco/Run_1_20_08_23_12_09', "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [ ]:
# Call COCO Evaluator function and pass the Test Dataset
evaluator = COCOEvaluator("building_test_coco", cfg, False, output_dir=f"{cfg.OUTPUT_DIR}/test/")
val_loader = build_detection_test_loader(cfg, "building_test_coco")

inference_on_dataset(predictor.model, val_loader, evaluator)

In [ ]:
# Read and predict an image
image = cv2.imread('/home/ubuntu/data/unzipped/facade-original-coco-segmentation/valid/IMG_5868_scaled_0_png_jpg.rf.1df4f5078028640ec94e983240d9521a.jpg')
outputs = predictor(image)

# Visualize the results
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize=(10,10))
plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
# List of image paths
image_paths = [
    '/home/ubuntu/data/unzipped/facade-original-coco-segmentation/valid/IMG_5868_scaled_0_png_jpg.rf.1df4f5078028640ec94e983240d9521a.jpg',
    '/home/ubuntu/data/unzipped/facade-original-coco-segmentation/valid/IMG_6044_scaled_0_png_jpg.rf.07fdb382874e1f14f00044f0f88f2989.jpg',
    '/home/ubuntu/data/unzipped/facade-original-coco-segmentation/valid/IMG_6153_scaled_0_png_jpg.rf.5933f74b904a718082ba7968e02128c1.jpg',
    '/home/ubuntu/data/unzipped/facade-original-coco-segmentation/valid/IMG_6002_scaled_0_png_jpg.rf.19cbe00a2856981e0bbac9dadd6bf3cc.jpg'
]

# Predict all images
outputs_list = [predictor(cv2.imread(image_path)) for image_path in image_paths]

plt.figure(figsize=(25,25))

for i, (image_path, outputs) in enumerate(zip(image_paths, outputs_list)):
    image = cv2.imread(image_path)
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    plt.subplot(2, 2, i+1)  # 2x2 grid for 4 images
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.axis('off')  # hide axes for clarity

plt.tight_layout()
plt.show()